In [1]:
from matplotlib import image
import numpy as np
import matplotlib.pyplot as plt



class fft:
    def __init__(self, image):
        self.M = image.shape[0]
        self.N = image.shape[1]

    def fft1d(self, data):
        N = len(data)
        if N <= 1: return data
        even = self.fft1d(data[0::2])
        odd =  self.fft1d(data[1::2])
        T= [np.exp(-2j*np.pi*k/N)*odd[k] for k in range(N//2)]
        return [even[k] + T[k] for k in range(N//2)] + \
           [even[k] - T[k] for k in range(N//2)]
    
    
    def image_reshape(self,image):
        M = image.shape[0]
        N = image.shape[1]
        i = 0
        j = 0
        while(M>1):
            M = M/2
            i = i+1
        
        while(N>1):
            N = N/2
            j+=1
        
        if M==1  and N==1:
            return image

        else:
            k = (2)**i -image.shape[0]          #raws to be added
            l = (2)**j - image.shape[1]         #column to be added

            #create empty matrix
            mat_c = np.zeros([image.shape[0], l], dtype= np.int16)
            image_re = np.append(image,mat_c, axis=1)

            mat_r = np.zeros([k, image_re.shape[1]], dtype= np.int16)
            image_re = np.append(image_re,mat_r, axis = 0)

            return image_re
 
    def fft2d(self,x):
        image = self.image_reshape(x)
        M = image.shape[0]
        N = image.shape[1]
        k=np.zeros([M,N],dtype=np.complex128)
        l = np.zeros([M,N],dtype=np.complex128)
        for i in range(N):
            k[:,i]= self.fft1d(image[:,i])
        for j in range(M):
            l[j,:] = self.fft1d(k[j,:])

        return l
    
    def shifting(self, data):
        M = data.shape[0]
        N = data.shape[1]
        output = np.zeros([M,N],dtype=np.complex128)
        for i in range(M):
            for j in range(N):
                h = data[i][j]
                if i<=(M//2-1) and j <(N//2 -1):
                    i = i + (M//2)
                    j= j+ (N//2)
        
                elif i > (M//2 - 1) and j <= (N//2 -1):
                    i = i- (M//2)
                    j = j+ (N//2)
        
                elif i <= (M//2-1) and j >(N//2-1):
                    i = i + (M//2)
                    j= j -(N//2)

                else:
                    i= i - (M//2)
                    j= j - (N//2)
        
                output[i][j] = h

        return output

    def ifft2d(self,x):
        x = np.conj(x)
        s = self.fft2d(x)

        return (self.M*self.N)*np.conj(s)[0:self.M,0:self.N]
